In [1]:
import pandas as pd
from gensim.models.keyedvectors import KeyedVectors

In [2]:
wv_skipgram_w5 = KeyedVectors.load_word2vec_format("results/models/m_skipgram_w5.vec")
wv_skipgram_w10 = KeyedVectors.load_word2vec_format("results/models/m_skipgram_w10.vec")
wv_cbow_w5 = KeyedVectors.load_word2vec_format("results/models/m_cbow_w5.vec")

# https://fasttext.cc/docs/en/crawl-vectors.html#models
wv_pretrained = KeyedVectors.load_word2vec_format("results/models/cc.be.300.vec")

In [3]:
def apply_func(func, index = None):
    map_func = lambda x: ": ".join(map(str,x)) if type(x) == tuple else x
    
    table = {
        "skipgram w5": map(map_func, func(wv_skipgram_w5, "skipgram w5")),
        "skipgram w10": map(map_func, func(wv_skipgram_w10, "skipgram w10")),
        "cbow w5": map(map_func, func(wv_cbow_w5, "cbow w5")),
        "pretrained": map(map_func, func(wv_pretrained, "pretrained"))
    }
    
    return pd.DataFrame(table, index)

In [4]:
apply_func(
    lambda x, _: [len(x.vectors)],
    index=["Number of vectors"]
)

,skipgram w5,skipgram w10,cbow w5,pretrained
Number of vectors,114491,114491,114491,1714011


# Outliers detection

In [5]:
outliers_stats = {
    "skipgram w5": 0,
    "skipgram w10": 0,
    "cbow w5": 0,
    "pretrained": 0
}


def detect_outliers(wv: KeyedVectors, model_name: str):
    column = []
    
    for outlier in cluster_outliers:
        predicted_outlier = wv.doesnt_match(
            cluster_elements + [outlier]
        )
        
        if predicted_outlier == outlier:
            column.append("+")
        else:
            column.append("- " + predicted_outlier)
            
    column.append(f"{column.count('+')}/8")
    outliers_stats[model_name] += column.count('+')
    
    return column

In [6]:
cluster_elements = ["чырвоны", "белы", "сіні", "блакітны", "зялёны", "чорны", "жоўты", "карычневы"]
cluster_outliers = ["мастак", "вясёлка", "гарбата", "аловак", "неба", "плот", "эмоцыя", "старажытнасць"]

apply_func(detect_outliers, index=cluster_outliers + ["Correct predictions"])

,skipgram w5,skipgram w10,cbow w5,pretrained
мастак,+,+,+,+
вясёлка,- белы,- чырвоны,+,+
гарбата,+,- чырвоны,+,+
аловак,+,+,+,+
неба,+,+,+,+
плот,+,+,+,+
эмоцыя,+,+,+,+
старажытнасць,+,+,+,+
Correct predictions,7/8,6/8,8/8,8/8


In [7]:
cluster_elements = ["сабака", "кот", "леў", "мыш", "ліс", "воўк", "мядзведзь", "заяц"]
cluster_outliers = ["лес", "камень", "пыса", "хвост", "заапарк", "зграя", "тэхніка", "мастацтва"]

apply_func(detect_outliers, index=cluster_outliers + ["Correct predictions"])

,skipgram w5,skipgram w10,cbow w5,pretrained
лес,+,+,+,- мыш
камень,+,+,+,+
пыса,- сабака,- сабака,- сабака,- мыш
хвост,+,+,- сабака,- мыш
заапарк,- сабака,+,- сабака,- мыш
зграя,- сабака,+,- сабака,- мыш
тэхніка,+,+,+,+
мастацтва,+,+,+,+
Correct predictions,5/8,7/8,4/8,3/8


In [8]:
cluster_elements = ["студзень", "сакавік", "май", "ліпень", "жнівень", "верасень", "лістапад", "снежань"]
cluster_outliers = ["серада", "зіма", "дата", "год", "астралогія", "нараджэнне", "мяч", "папера"]

apply_func(detect_outliers, index=cluster_outliers + ["Correct predictions"])

,skipgram w5,skipgram w10,cbow w5,pretrained
серада,+,+,+,+
зіма,+,+,+,+
дата,+,+,+,+
год,+,+,+,+
астралогія,+,+,+,+
нараджэнне,+,+,+,+
мяч,+,+,+,+
папера,+,+,+,+
Correct predictions,8/8,8/8,8/8,8/8


In [9]:
cluster_elements = ["галава", "рука", "нага", "тулава", "шыя", "палец", "спіна", "плячо"]
cluster_outliers = ["медыцына", "мышца", "шапка", "адзенне", "футбол", "праца", "смоўж", "планета"]

apply_func(detect_outliers, index=cluster_outliers + ["Correct predictions"])

,skipgram w5,skipgram w10,cbow w5,pretrained
медыцына,+,+,+,+
мышца,- галава,- галава,+,- палец
шапка,- галава,- галава,- галава,+
адзенне,- галава,+,+,+
футбол,+,+,+,+
праца,+,+,+,+
смоўж,- галава,+,+,+
планета,+,+,+,+
Correct predictions,4/8,6/8,7/8,7/8


In [10]:
cluster_elements = ["масленіца", "купалле", "вялікдзень", "спас", "сёмуха", "вадохрышча", "радаўніца", "дажынкі"]
cluster_outliers = ["падарунак", "святочнасць", "абрад", "прымха", "прысмак", "сузор'е", "графін", "воск"]

apply_func(detect_outliers, index=cluster_outliers + ["Correct predictions"])

,skipgram w5,skipgram w10,cbow w5,pretrained
падарунак,+,+,- вадохрышча,+
святочнасць,+,+,- вадохрышча,- спас
абрад,+,+,- вадохрышча,- спас
прымха,- вадохрышча,- спас,- вадохрышча,+
прысмак,+,+,- вадохрышча,- спас
сузор'е,- вадохрышча,- спас,- вадохрышча,- спас
графін,- вадохрышча,- спас,- вадохрышча,+
воск,+,- спас,- вадохрышча,+
Correct predictions,5/8,4/8,0/8,4/8


In [11]:
cluster_elements = ["вуліца", "дом", "парк", "плошча", "праспект", "сквер", "завулак", "бульвар"]
cluster_outliers = ["вёска", "краіна", "мэр", "транспарт", "турыст", "мапа", "кава", "крыж"]

apply_func(detect_outliers, index=cluster_outliers + ["Correct predictions"])

,skipgram w5,skipgram w10,cbow w5,pretrained
вёска,+,+,+,+
краіна,+,+,+,+
мэр,- дом,- дом,- дом,+
транспарт,+,+,- дом,+
турыст,- дом,- вуліца,- дом,+
мапа,+,+,- дом,+
кава,+,+,+,+
крыж,+,+,+,+
Correct predictions,6/8,6/8,4/8,8/8


In [12]:
cluster_elements = ["аўтобус", "тралейбус", "машына", "трамвай", "метро", "ровар", "самакат", "матацыкл"]
cluster_outliers = ["дарога", "кола", "горад", "квіток", "маршрут", "хуткасць", "груша", "месяц"]

apply_func(detect_outliers, index=cluster_outliers + ["Correct predictions"])

,skipgram w5,skipgram w10,cbow w5,pretrained
дарога,+,+,- самакат,+
кола,+,+,- самакат,+
горад,+,+,- самакат,+
квіток,- машына,+,- самакат,- метро
маршрут,- машына,+,- самакат,- метро
хуткасць,+,+,- самакат,+
груша,+,+,- самакат,+
месяц,+,+,- самакат,+
Correct predictions,6/8,8/8,0/8,6/8


In [13]:
cluster_elements = ["рондаль", "патэльня", "чайнік", "духоўка", "пліта", "тарка", "халадзільнік", "скавародка"]
cluster_outliers = ["страва", "кухар", "пакой", "вада", "кватэра", "канапа", "марс", "трава"]

apply_func(detect_outliers, index=cluster_outliers + ["Correct predictions"])

,skipgram w5,skipgram w10,cbow w5,pretrained
страва,+,+,- духоўка,- пліта
кухар,+,+,- духоўка,+
пакой,+,+,- духоўка,+
вада,+,+,- духоўка,+
кватэра,+,+,- духоўка,+
канапа,+,+,- духоўка,- пліта
марс,- пліта,- пліта,- духоўка,+
трава,+,+,- духоўка,+
Correct predictions,7/8,7/8,0/8,6/8


In [14]:
for col, val in outliers_stats.items():
    outliers_stats[col] = [f"{val}/64", val / 64 * 100]

pd.DataFrame(
    data=outliers_stats,
    index=["Number of correctly predicted", "% of correctly predicted"]
).T

,Number of correctly predicted,% of correctly predicted
skipgram w5,48/64,75.0
skipgram w10,52/64,81.25
cbow w5,31/64,48.4375
pretrained,50/64,78.125


# Analysis

#### Similarity to one word

In [15]:
apply_func(
    lambda x, _: x.most_similar(["маці"], topn=20)
)

,skipgram w5,skipgram w10,cbow w5,pretrained
0,бацька: 0.8766636252403259,бацька: 0.8542380332946777,бацька: 0.8858320713043213,маці-немкі: 0.7341005802154541
1,жонка: 0.8560835719108582,матка: 0.8250699639320374,жонка: 0.8673643469810486,маці-і-мачахі: 0.7307584285736084
2,нявестка: 0.8346813321113586,унучка: 0.8138112425804138,сын: 0.8077619075775146,маці.-: 0.726932168006897
3,матка: 0.8244144916534424,нявестка: 0.8107252717018127,матка: 0.7976338267326355,Шашандухт: 0.6899331212043762
4,унучка: 0.8112714886665344,дачка: 0.7857580184936523,дзіця: 0.7898380756378174,Сандухт: 0.6643711924552917
5,дачка: 0.8096562623977661,дачушка: 0.7850747108459473,дачка: 0.7854682803153992,бацька-маці: 0.6442077159881592
6,Светка: 0.8096061944961548,брацік: 0.7834899425506592,сястра: 0.756388783454895,Мюмлы-маці: 0.6388117671012878
7,мама: 0.806246280670166,жонка: 0.7790187001228333,дзед: 0.7472432255744934,бацька: 0.636451005935669
8,брацік: 0.8024640083312988,цётка: 0.7726487517356873,малы: 0.7416197061538696,Багхмаці: 0.6317927241325378
9,сын: 0.8007441163063049,сын: 0.7709203362464905,муж: 0.7351961731910706,Гурандухт: 0.6307734847068787


In [16]:
apply_func(
    lambda x, _: x.most_similar(["вучоны"], topn=20)
)

,skipgram w5,skipgram w10,cbow w5,pretrained
0,партыец: 0.8713955879211426,крытык: 0.8684261441230774,работнік: 0.9049365520477295,невучоны: 0.7828152179718018
1,крытык: 0.8697142004966736,дзеяч: 0.8519879579544067,пісьменнік: 0.904116690158844,вучоны-фiзiк: 0.7132922410964966
2,пісьменнік: 0.869478166103363,правакацыя: 0.8459656238555908,паэт: 0.8982222676277161,вучоны-медык: 0.7085844874382019
3,акадэмік: 0.8548937439918518,таленавіты: 0.8400018811225891,бальшавік: 0.8926443457603455,вучоны-хiмiк: 0.6963252425193787
4,празаік: 0.8547191619873047,дэтэктыў: 0.8394523859024048,адукацыя: 0.890635073184967,савецкі-вучоны: 0.6960341930389404
5,дзеяч: 0.8544794917106628,класік: 0.8388006091117859,дзеяч: 0.8903725147247314,вучоны-езуiт: 0.6909222602844238
6,інтэлігенцыя: 0.8540493845939636,папулярны: 0.8384833931922913,аўтар: 0.8873587250709534,вучоны-езуіт: 0.6841995716094971
7,таленавіты: 0.8507794737815857,журналіст: 0.836736261844635,эмігрант: 0.8820002675056458,вучоны-імунолаг: 0.6814020872116089
8,інтэлігент: 0.8494454622268677,партыец: 0.8356762528419495,агент: 0.8790002465248108,вучоны-фiлосаф: 0.6709549427032471
9,крамольнік: 0.848173975944519,празаік: 0.8336595296859741,інтэлігенцыя: 0.8749860525131226,вучоны-славіст: 0.6671708226203918


In [17]:
apply_func(
    lambda x, _: x.most_similar(["прыгожы"], topn=20)
)

,skipgram w5,skipgram w10,cbow w5,pretrained
0,інтэлігентны: 0.7538114786148071,убранне: 0.7538516521453857,сціплы: 0.7470298409461975,прыгожыю: 0.7629463076591492
1,прывабны: 0.7493212819099426,прычоска: 0.7517766356468201,дамавітаваць: 0.7358946204185486,непрыгожы: 0.756121814250946
2,харошы: 0.7461376786231995,гожы: 0.750282347202301,вясёлы: 0.735693097114563,Непрыгожы: 0.7520151734352112
3,убранне: 0.7410526871681213,сціплы: 0.7488327026367188,надзіва: 0.7283241748809814,прыгожымi: 0.733957827091217
4,прычоска: 0.7388331890106201,прыгажуня: 0.7451572418212891,суровы: 0.7261638641357422,прыгожым.: 0.7322680354118347
5,сціплы: 0.737297534942627,сінявокі: 0.7305049896240234,светлы: 0.7260852456092834,прыгож: 0.6918627619743347
6,светлавалосы: 0.7347528338432312,сімпатычны: 0.7296237349510193,гожы: 0.7202317118644714,прыгож-: 0.686297595500946
7,гожы: 0.7318159937858582,ганарысты: 0.72943514585495,маладжавы: 0.7151607275009155,прыгожу: 0.6600285768508911
8,сімпатычны: 0.7300323843955994,усмешлівы: 0.7265523076057434,далікатны: 0.7142751812934875,прыгожа.: 0.6257501840591431
9,непрыгожы: 0.7264258861541748,маладжавы: 0.7258691787719727,хударлявы: 0.7121385931968689,Прыгожы: 0.6182975172996521


In [18]:
apply_func(
    lambda x, _: x.most_similar(["адзінаццаць"], topn=20)
)

,skipgram w5,skipgram w10,cbow w5,pretrained
0,шаснаццаць: 0.9279336929321289,чатырнаццаць: 0.8911280035972595,дзевяць: 0.9566066861152649,адзінаццацю: 0.701328456401825
1,васемнаццаць: 0.9275397062301636,семнаццаць: 0.8886259198188782,сорак: 0.9525117874145508,наццаць: 0.6992630958557129
2,чатырнаццаць: 0.9269293546676636,гок: 0.8833463191986084,восем: 0.937562882900238,Адзінаццаць: 0.6875142455101013
3,дзевяць: 0.925966203212738,дзевяць: 0.8822671175003052,семнаццаць: 0.9298513531684875,двенаццаць: 0.663983166217804
4,семнаццаць: 0.9246982932090759,праслужыць: 0.8762643933296204,васемнаццаць: 0.9246838092803955,мнаццаць: 0.6573565602302551
5,чатырыста: 0.919135570526123,восьмы: 0.8753341436386108,пятнаццаць: 0.9217433929443359,Шаснаццаць: 0.6406635642051697
6,трынаццаць: 0.9149125218391418,васемнаццаць: 0.8729905486106873,шаснаццаць: 0.9208481311798096,дзвенаццаць: 0.6289210915565491
7,адзінаццаты: 0.9129078388214111,шосты: 0.8728602528572083,дванаццаць: 0.9149121642112732,пяць: 0.6240526437759399
8,сорак: 0.9119332432746887,шаснаццаць: 0.8696613907814026,паўтара: 0.9143983721733093,чатырнаццаць: 0.621525764465332
9,дзевяноста: 0.9116633534431458,трынаццаць: 0.8692251443862915,трыццаць: 0.9134524464607239,семнаццаць: 0.6173511743545532


In [19]:
apply_func(
    lambda x, _: x.most_similar(["ён"], topn=20)
)

,skipgram w5,skipgram w10,cbow w5,pretrained
0,яна: 0.9091382622718811,яна: 0.8612225651741028,яна: 0.8698540329933167,Ён: 0.7677587270736694
1,Наўмыснік: 0.8739092946052551,і: 0.8575137853622437,Мацвей: 0.7059121131896973,не: 0.5042052865028381
2,Паліна: 0.8664207458496094,што: 0.8252401351928711,яны: 0.7013801336288452,",: 0.4746936559677124"
3,тварыцкі: 0.8617824912071228,не: 0.8073900938034058,Лабановіч: 0.6072115302085876,Ен: 0.4343029856681824
4,Чарпакевіч: 0.8601926565170288,Бурачок: 0.8063921928405762,Алесь: 0.5904755592346191,ен: 0.40082284808158875
5,Балоціч: 0.8591228723526001,Алесь: 0.8053411245346069,Ганна: 0.5796592831611633,Віарт: 0.39620304107666016
6,мар’янін: 0.857450008392334,Балоціч: 0.8001131415367126,я: 0.5764326453208923,КалЁ: 0.39555639028549194
7,Вольза: 0.8556194305419922,звычны: 0.798690676689148,Башлыкоў: 0.5748684406280518,жа: 0.3915726840496063
8,Камбрыг: 0.8552753925323486,пачутаць: 0.7986211180686951,Славік: 0.5716134309768677,бы: 0.3907463550567627
9,Чумак: 0.8541934490203857,прагнагчы: 0.7963274121284485,Чубар: 0.5691579580307007,ссівеў: 0.3873329758644104


In [20]:
apply_func(
    lambda x, _: x.most_similar(["хутка"], topn=20)
)

,skipgram w5,skipgram w10,cbow w5,pretrained
0,неўзабаве: 0.7111973762512207,неўзабаве: 0.7135466933250427,неўзабаве: 0.6196990609169006,хутка-хутка: 0.7617853879928589
1,шпарка: 0.7094509601593018,прагнагчы: 0.7004532217979431,марудна: 0.5777590274810791,няхутка: 0.6932395696640015
2,найхутчэй: 0.7062945365905762,найхутчэй: 0.6879240274429321,неяк: 0.5581346154212952,Няхутка: 0.6723303198814392
3,борзда: 0.7050652503967285,удала: 0.6867664456367493,нечакана: 0.541629433631897,ціхутка: 0.6073318719863892
4,імгненна: 0.7049323916435242,няхутка: 0.6818335652351379,непрыкметна: 0.539616048336029,добрахвотна: 0.5923420190811157
5,шпарчэй: 0.6883629560470581,своечасова: 0.6803925037384033,наперад: 0.5394963622093201,вельмі: 0.5869695544242859
6,спакваля: 0.6842737197875977,карцець: 0.6802246570587158,далей: 0.5334013104438782,паволі: 0.5852990746498108
7,удала: 0.6838054656982422,цярпецца: 0.6761101484298706,памалу: 0.5309447050094604,Павольна: 0.5753752589225769
8,няхутка: 0.6835536956787109,шпарка: 0.6739299297332764,шпарка: 0.5306515097618103,хутказ: 0.5725973844528198
9,дахаты: 0.6819209456443787,ненадоўга: 0.6734201908111572,нарэшце: 0.5300918817520142,паступова: 0.5664605498313904


#### Arithmetic operations with multiple vectors

In [21]:
apply_func(
    lambda x, _: x.most_similar(positive=['жанчына', 'бацька'], negative=['мужчына'], topn=20)
)

,skipgram w5,skipgram w10,cbow w5,pretrained
0,маці: 0.8194512724876404,маці: 0.8091663718223572,маці: 0.8502755761146545,маці: 0.6002719402313232
1,сын: 0.7707741856575012,жонка: 0.765314519405365,жонка: 0.8084573745727539,бацька-маці: 0.5851595401763916
2,муж: 0.760155975818634,сын: 0.7572315335273743,сын: 0.805460512638092,-бацька: 0.5826510787010193
3,нявестка: 0.7424349188804626,Саша: 0.7569324970245361,дачка: 0.7420547604560852,бацька.-: 0.580398440361023
4,дачка: 0.7418791651725769,нявестка: 0.7457756400108337,муж: 0.7357462644577026,бацька-эстонец: 0.5780854821205139
5,брацік: 0.7253528237342834,маткі: 0.7432278990745544,матка: 0.6798509955406189,ваенны-бацька: 0.5651611685752869
6,сяброўка: 0.7231695652008057,сірата: 0.7395504713058472,сястра: 0.6788142323493958,бацька-2: 0.5649682283401489
7,жонка: 0.7196933627128601,дачка: 0.7344971299171448,дзіця: 0.6783962249755859,Бог-бацька: 0.5583592653274536
8,унучка: 0.7158250212669373,ахмістрыня: 0.7326514720916748,дзед: 0.6736637353897095,бог-бацька: 0.5582204461097717
9,Алена: 0.7086225748062134,унучка: 0.7316990494728088,Ніна: 0.6228703260421753,бацькава: 0.5555803179740906


In [22]:
apply_func(
    lambda x, _: x.most_similar(positive=['жанчына', 'хлопец'], negative=['мужчына'], topn=20)
)

,skipgram w5,skipgram w10,cbow w5,pretrained
0,дзяўчына: 0.8390379548072815,дзяўчына: 0.7680259943008423,дзяўчына: 0.8158063888549805,хлопец.-: 0.6098657846450806
1,кабет: 0.7205139398574829,Валянціна: 0.7267442941665649,сябар: 0.6204489469528198,хлопецъ: 0.5886616110801697
2,хлопчык: 0.7160747647285461,Саша: 0.7246918678283691,чалавек: 0.6101166605949402,дзяўчына: 0.5662907361984253
3,Хрысцінюк: 0.7113121151924133,Аня: 0.7138403058052063,хлопчык: 0.5999425053596497,хлопе: 0.5164709687232971
4,салдатка: 0.7070057392120361,яна: 0.7106106877326965,баец: 0.5978458523750305,хлопае: 0.4926866590976715
5,Сандро: 0.7067594528198242,Лена: 0.7055127024650574,Міленькі: 0.5949402451515198,.Хлопец: 0.48046717047691345
6,яна: 0.7043850421905518,суседак: 0.7012845277786255,гаспадыня: 0.5868555307388306,Хлопец-хлапчук: 0.4693601131439209
7,Вікберт: 0.7019421458244324,дураслівы: 0.6987898349761963,кабет: 0.5854188799858093,хлоп: 0.46652278304100037
8,Ліда: 0.6998673677444458,удзячна: 0.6928006410598755,дзіця: 0.5845623016357422,хлопнуў: 0.4653664231300354
9,дзяўчынка: 0.6977933645248413,Буткевіч: 0.6919383406639099,настаўнік: 0.5827525854110718,Хлопец: 0.4651722013950348


In [23]:
apply_func(
    lambda x, _: x.most_similar(positive=['жанчына', 'прадавец'], negative=['мужчына'], topn=20)
)

,skipgram w5,skipgram w10,cbow w5,pretrained
0,нежаданне: 0.8022024035453796,далікатнасць: 0.80511873960495,скарга: 0.8359129428863525,прадавачка: 0.562881588935852
1,інтымны: 0.8012632727622986,уздзейнічаць: 0.7882839441299438,латыш: 0.824915885925293,прадаўшчыца: 0.5403940677642822
2,здатнасць: 0.8001471757888794,бібліятэкарка: 0.7863178849220276,пярэчанне: 0.8233468532562256,Прадавец: 0.5330600738525391
3,няўважлівы: 0.7981935739517212,павучальны: 0.78294837474823,рэпліка: 0.8228441476821899,прадава: 0.5251862406730652
4,слушны: 0.7981559038162231,падрыхтоўваць: 0.7820546627044678,Грамыка: 0.8225812315940857,прадажная: 0.5108407735824585
5,талкова: 0.797240138053894,эфект: 0.7820437550544739,покліч: 0.8215352296829224,прадаю: 0.4973801076412201
6,Дубейка: 0.796680748462677,іншаземец: 0.7817070484161377,Мар'ян: 0.819329023361206,прадауца: 0.49516016244888306
7,Шульцэ: 0.7958831191062927,Дарошка: 0.7799788117408752,Дамецік: 0.8170785903930664,прадавала: 0.48551368713378906
8,таварыскасць: 0.7957931756973267,відочна: 0.7790292501449585,дамініканец: 0.8169015049934387,прадаж: 0.47671884298324585
9,палітрука: 0.7948907613754272,імяніна: 0.7788580060005188,Кудравец: 0.8146253228187561,прадавачак: 0.4718766510486603
